# GRE
![](gre.png)

### Step 1 : Configure Interfaces 

```
!r1
conf t
	int f0/0
		ip add  12.0.0.1 255.255.255.0
		no sh
	int l0
		ip add  192.168.1.1 255.255.255.0
	int l1
		ip add  10.1.1.1 255.255.255.0
	int l2
		ip add  10.1.2.1 255.255.255.0
	int l3
		ip add  10.1.3.1 255.255.255.0
end

!r2
conf t
	int f0/0
		ip add  12.0.0.2 255.255.255.0
		no sh
	int f0/1
		ip add  23.0.0.2 255.255.255.0
		no sh
	int l0
		ip add  192.168.2.1 255.255.255.0
end

!r3
conf t
	int f0/0
		ip add  23.0.0.3 255.255.255.0
		no sh
	int l0
		ip add  192.168.3.1 255.255.255.0
	int l1
		ip add  10.3.1.1 255.255.255.0
	int l2
		ip add  10.3.2.1 255.255.255.0
	int l3
		ip add  10.3.3.1 255.255.255.0
end
```

### Step 2 : Configure Underlay routing protocol (BGP)
```
!r1 
conf t
    !bgp oconfig (undelay) 
	router bgp 100
		no auto
		no syn

		nei 12.0.0.2 remote-as 200
		nei 12.0.0.2 version 4
		nei 12.0.0.2 password cisco

		net 192.168.1.0 mask 255.255.255.0
end

!r2
conf t
	!bgp oconfig (undelay) 
	router bgp 200
		no auto
		no syn

		nei 12.0.0.1 remote-as 100
		nei 12.0.0.1 version 4
		nei 12.0.0.1 password cisco

		nei 23.0.0.3 remote-as 300
		nei 23.0.0.3 version 4
		nei 23.0.0.3 password cisco

		net 192.168.2.0 mask 255.255.255.0
end 

!r3
conf t
    !bgp oconfig (undelay) 
	router bgp 300
		no auto
		no syn

		nei 23.0.0.2 remote-as 200
		nei 23.0.0.2 version 4
		nei 23.0.0.2 password cisco

		net 192.168.3.0 mask 255.255.255.0
end 
```
### Step 3 : Create a Bidirectional Tunnel
```
!r1
conf t
	!tunnel config 
	int tunnel 1                       ! add  virtual tunnel interface
		ip add 3.3.3.1 255.255.255.0   ! set ip address of tunnel
		tunnel sour 192.168.1.1        ! set source interface or IP address
		tunnel dest 192.168.3.1        ! set destination int or IP address
end

!r3
conf t
    !tunnel config 
	int tunnel 1
		ip add 3.3.3.2 255.255.255.0
		tunnel sour 192.168.3.1 
		tunnel dest 192.168.1.1
end
```

### Step 4 : Configure Overlay routing (EIGRP)
```
!r1
conf t
	!eigrp config (overlay)
	router eigrp 10
		no auto 
		passive def

		net 3.3.3.1 0.0.0.0
		net 10.1.1.1 0.0.0.0
		net 10.1.2.1 0.0.0.0
		net 10.1.3.1 0.0.0.0

		no passive tunnel 1
end 

!r3
conf t
	!eigrp config (overlay)
	router eigrp 10
		no auto 
		passive def

		net 3.3.3.2 0.0.0.0
		net 10.3.1.1 0.0.0.0
		net 10.3.2.1 0.0.0.0
		net 10.3.3.1 0.0.0.0

		no passive tunnel 1
end 


```

# DMVPN
![](dmvpn_gre.png)

### Step 1: intererface config 
```
!hub
conf t
    int f0/0
        ip add 123.0.0.100 255.255.255.0
        no sh
    int l0
        ip add 192.168.1.100 255.255.255.0
end
        
!spoke1
conf t
    int f0/0
        ip add 123.0.0.1 255.255.255.0
        no sh 
    int l0
        ip add 192.168.1.1 255.255.255.0
    int l1
        192.168.10.1 255.255.255.0
    int l2
        192.168.11.1 255.255.255.0
    int l3
        192.168.12.1 255.255.255.0
end 
        

!spoke2
conf t
    int f0/0
        ip add 123.0.0.2 255.255.255.0
        no sh 
    int l0
        ip add 192.168.2.1 255.255.255.0
    int l1
        192.168.20.1 255.255.255.0
    int l2
        192.168.21.1 255.255.255.0
    int l3
        192.168.22.1 255.255.255.0
end 
        
!spoke3
conf t
    int f0/0
        ip add 123.0.0.3 255.255.255.0
        no sh 
    int l0
        ip add 192.168.3.1 255.255.255.0
    int l1
        192.168.30.1 255.255.255.0
    int l2
        192.168.31.1 255.255.255.0
    int l3
        192.168.32.1 255.255.255.0
end
        
```

### Step 2: Overlay Routing (BGP)
```
!hub
conf t
    router bgp 1000
        no synchronization
        no auto-summary
        
        neighbor 123.0.0.1 remote-as 100
        neighbor 123.0.0.1 password cisco
        
        neighbor 123.0.0.2 remote-as 200
        neighbor 123.0.0.2 password cisco
        
        neighbor 123.0.0.3 remote-as 300
        neighbor 123.0.0.3 password cisco
end 

!spoke1
conf t
    router bgp 100
        no synchronization
        no auto-summary
        
        neighbor 123.0.0.100 remote-as 1000
        neighbor 123.0.0.100 password cisco
 
        network 192.168.1.0
end

!spoke2
conf t
    router bgp 100
        no synchronization
        no auto-summary
        
        neighbor 123.0.0.100 remote-as 1000
        neighbor 123.0.0.100 password cisco
 
        network 192.168.2.0
end

!spoke3
conf t
    router bgp 100
        no synchronization
        no auto-summary
        
        neighbor 123.0.0.100 remote-as 1000
        neighbor 123.0.0.100 password cisco
 
        network 192.168.3.0
end
```

### Step 3 : Configure DMVPN Tunnel
```
!hub
conf t
    interface Tunnel1
         ip address 10.0.0.100 255.255.255.0
         tunnel source FastEthernet0/0
         tunnel mode gre multipoint           ! set mGRE mode
         
         ip nhrp network-id 1                 ! grup ID
         ip nhrp map multicast dynamic        ! to accommodate eigrp multicast
         
         no ip next-hop-self eigrp 10
         no ip split-horizon eigrp 10         ! for eigrp
end

!spokes
conf t
    interface Tunnel1
        ip address 10.0.0.1 255.255.255.0
        tunnel source FastEthernet0/0
        tunnel mode gre multipoint
         
        ip nhrp network-id 1                  ! match group ID
        ip nhrp nhs 10.0.0.100                ! specify NHRP server (hub)
        ip nhrp map 10.0.0.100 123.0.0.100    ! map NHS tun : phy addr
        ip nhrp map multicast dynamic         ! for eigrp 
end
```



### Step 4 : Overlay Routing (EIGRP)
```
!hub
conf t
    router eigrp 10
         no auto-summary
         passive-interface default
         network 10.0.0.100 0.0.0.0
         no passive-interface Tunnel1
end

!spoke1
conf t
    router eigrp 10
         no auto-summary
         passive-interface default
         
         network 10.0.0.1 0.0.0.0
         network 192.168.10.1 0.0.0.0
         network 192.168.11.1 0.0.0.0
         network 192.168.12.1 0.0.0.0
 
         no passive-interface Tunnel1
end

!spoke2
conf t
    router eigrp 10
         no auto-summary
         passive-interface default
         
         network 10.0.0.2 0.0.0.0
         network 192.168.20.1 0.0.0.0
         network 192.168.21.1 0.0.0.0
         network 192.168.22.1 0.0.0.0
         
         no passive-interface Tunnel1
end

!spoke3
conf t
    router eigrp 10
         no auto-summary
         passive-interface default
         
         network 10.0.0.3 0.0.0.0
         network 192.168.30.1 0.0.0.0
         network 192.168.31.1 0.0.0.0
         network 192.168.32.1 0.0.0.0

         no passive-interface Tunnel1
end
```